In [24]:
# Install Earth Engine API
!pip install -q earthengine-api

In [25]:
import ee
ee.Authenticate()

True

In [26]:
ee.Initialize(project='deforestation-463506')

In [27]:
import datetime

# Define AOI: bounding box around Mabira Forest
region = ee.Geometry.Rectangle([32.9, 0.35, 33.1, 0.55])

# Time range
start = ee.Date('2018-01-01')
end   = ee.Date('2022-12-31')

# Monthly export parameters
num_patches = 2
patch_size = 128  # meters


In [28]:
def monthly_composite_labels(m):
    start_m = start.advance(m, 'month')
    end_m = start_m.advance(1, 'month')
    label = start_m.format('YYYY_MM')

    s2 = (ee.ImageCollection('COPERNICUS/S2_SR')
          .filterBounds(region)
          .filterDate(start_m, end_m)
          .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)))

    median = ee.Image(s2.median()).clip(region)

    # Spectral bands
    nir = median.select('B8')
    red = median.select('B4')
    green = median.select('B3')
    blue = median.select('B2')
    swir = median.select('B12')

    # Indices
    ndvi = nir.subtract(red).divide(nir.add(red)).rename('NDVI')
    evi = nir.subtract(red).multiply(2.5) \
          .divide(nir.add(red.multiply(6)).subtract(blue.multiply(7.5)).add(1)).rename('EVI')
    ndwi = green.subtract(nir).divide(green.add(nir)).rename('NDWI')
    nbr = nir.subtract(swir).divide(nir.add(swir)).rename('NBR')

    # Stack bands + indices
    composite = median.select(['B4', 'B3', 'B2']).addBands([ndvi, evi, ndwi, nbr])

    # Hansen forest loss mask
    gfc = ee.Image('UMD/hansen/global_forest_change_2024_v1_12')
    forest2000 = gfc.select('treecover2000').gt(30)
    lossYear = gfc.select('lossyear')

    year = ee.Number.parse(start_m.format('YYYY'))
    loss_condition = lossYear.gte(year.subtract(2000)).And(lossYear.lte(year.subtract(2000)))

    mask = forest2000.And(loss_condition).rename('forestMask').clip(region)

    return {'composite': composite, 'mask': mask, 'label': label}


In [29]:
# Export scheduled tasks
for m in range(60):  # 4 years × 12 months
    out = monthly_composite_labels(m)
    seeds = ee.FeatureCollection.randomPoints(region, num_patches, m)

    for i in range(num_patches):
        geom = ee.Feature(seeds.toList(num_patches).get(i)).geometry()
        id = out['label'].getInfo() + f'_{i+1}'

        # Image export
        ee.batch.Export.image.toDrive(
            image=out['composite'],
            description='IMG_' + id,
            folder='MabiraForestData3',
            fileNamePrefix='img_' + id,
            region=geom.bounds(),
            scale=10, maxPixels=1e9).start()

        # Corresponding mask export
        ee.batch.Export.image.toDrive(
            image=out['mask'],
            description='MASK_' + id,
            folder='MabiraForestData3',
            fileNamePrefix='mask_' + id,
            region=geom.bounds(),
            scale=10, maxPixels=1e9).start()

print(" 🚀 Exports started! You can monitor tasks at code.earthengine.google.com/tasks")


 🚀 Exports started! You can monitor tasks at code.earthengine.google.com/tasks
